Gurobi建模过程：
1. 建立全部的变量
2. 设立好目标函数
3. 建立好约束条件
4. 运行优化器

Gurobi tuplelist和tupledict的使用，主要是为了使用$x_{ij}$这类形式。


Gurobi建模举例1：
$$\max x+y+2z$$
$$ s.t. x+2y+3z \le 4$$
$$    x+y\ge 1$$
$$ x,y, z \in \{0,1\}$$

In [5]:
import gurobipy
try:
    m = gurobipy.Model('model_ex1')
    x = m.addVar(vtype=gurobipy.GRB.BINARY, name='x')
    y = m.addVar(vtype=gurobipy.GRB.BINARY, name='y')
    z = m.addVar(vtype=gurobipy.GRB.BINARY, name='z')

    m.setObjective(x + y + 2 * z, gurobipy.GRB.MAXIMIZE)
    m.addConstr(z + 2 * y + 3 * z <= 4, 'c0')
    m.addConstr(x + y >= 1, 'c1')
    m.optimize()
    for v in m.getVars():
        print('{} {}'.format(v.varName, v.x))
    print('obj: {}'.format(m.objVal))

except gurobipy.GurobiError as e:
    print('Error Code' + str(e.errno) + ":" + str(e))
    
except AttributeError:
    print('Encountered an attribute error')

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 2 rows, 3 columns and 4 nonzeros
Model fingerprint: 0x5bddebf2
Variable types: 0 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 2 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 4 available processors)

Solution count 2: 3 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%
x 1.0
y 0.0
z 1.0
obj: 3.0


营养配方模型
* 四种营养 calories, protein, fat, sodium
* 食物来源 hamburger, chicken, hot dog, fries, macaroni, pizza, salad, milk, ice cream
* 营养吸收有上限和下限
* 食物价格不同
* 食物营养不同

In [4]:
import gurobipy
categories, minNutrition, maxNutrition=gurobipy.multidict({
    'calories':[1000,2000],
    'protein':[91, gurobipy.GRB.INFINITY],
    'fat':[0,65],
    'sodium':[0,1779]
})
foods, cost=gurobipy.multidict({
    'hamburger':2.49,
    'chicken':2.89,
    'hot dog':1.50,
    'fries':1.89,
    'macaroni':2.09,
    'pizza':1.99,
    'salad':2.49,
    'milk':0.89,
    'ice cream':1.59
})
nutritionValues = {
    ('hamburger', 'calories'): 410,
    ('hamburger', 'protein'):  24,
    ('hamburger', 'fat'):      26,
    ('hamburger', 'sodium'):   730,
    ('chicken',   'calories'): 420,
    ('chicken',   'protein'):  32,
    ('chicken',   'fat'):      10,
    ('chicken',   'sodium'):   1190,
    ('hot dog',   'calories'): 560,
    ('hot dog',   'protein'):  20,
    ('hot dog',   'fat'):      32,
    ('hot dog',   'sodium'):   1800,
    ('fries',     'calories'): 380,
    ('fries',     'protein'):  4,
    ('fries',     'fat'):      19,
    ('fries',     'sodium'):   270,
    ('macaroni',  'calories'): 320,
    ('macaroni',  'protein'):  12,
    ('macaroni',  'fat'):      10,
    ('macaroni',  'sodium'):   930,
    ('pizza',     'calories'): 320,
    ('pizza',     'protein'):  15,
    ('pizza',     'fat'):      12,
    ('pizza',     'sodium'):   820,
    ('salad',     'calories'): 320,
    ('salad',     'protein'):  31,
    ('salad',     'fat'):      12,
    ('salad',     'sodium'):   1230,
    ('milk',      'calories'): 100,
    ('milk',      'protein'):  8,
    ('milk',      'fat'):      2.5,
    ('milk',      'sodium'):   125,
    ('ice cream', 'calories'): 330,
    ('ice cream', 'protein'):  8,
    ('ice cream', 'fat'):      10,
    ('ice cream', 'sodium'):   180}
m=gurobipy.Model('diet')
buy=m.addVars(foods,name='buy')
m.setObjective(buy.prod(cost), gurobipy.GRB.MINIMIZE)

m.addConstrs((sum(nutritionValues[f, c] * buy[f] for f in foods)
             == [minNutrition[c], maxNutrition[c]]
             for c in categories), "_")
def printSolution():
    if m.status == gurobipy.GRB.OPTIMAL:
        print('\nCost: %g' % m.objVal)
        print('\nBuy:')
        buyx = m.getAttr('x', buy)
        for f in foods:
            print('%s %g' % (f, buyx[f]))
    else:
        print('No solution')


# Solve
m.optimize()
printSolution()

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 4 rows, 12 columns and 39 nonzeros
Model fingerprint: 0x47f21384
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [9e-01, 3e+00]
  Bounds range     [7e+01, 2e+03]
  RHS range        [7e+01, 2e+03]
Presolve removed 0 rows and 2 columns
Presolve time: 0.01s
Presolved: 4 rows, 10 columns, 37 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.222500e+02   0.000000e+00      0s
       3    9.6645465e+00   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.02 seconds
Optimal objective  9.664546521e+00

Cost: 9.66455

Buy:
hamburger 0
chicken 0
hot dog 0
fries 0
macaroni 0
pizza 0
salad 0.478961
milk 9.51903
ice cream 0
